In [1]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat", temperature=0.7)

上下文工程（Context Engineering）是构建基于大语言模型（LLM）应用时的一项核心设计实践，指的是有意识地设计、组织和优化输入给 LLM 的上下文内容（context），以引导模型生成更准确、可靠、安全且符合预期的输出。
你可以把它理解为 “给 AI 写提示词的艺术与科学”，但它的范围远不止于写一条 prompt，而是涵盖整个输入信息流的结构化管理。

# 上下文类型

| 上下文类型（Context Type） | 可以控制的内容 | 临时性 or 持久性 |
|--------------------------|----------------|------------------|
| 模型上下文（Model Context） | 输入到模型调用中的内容（指令、消息历史、工具列表、响应格式等） | 临时性（Transient） |
| 工具上下文（Tool Context） | 工具可以访问和生成的内容（对状态、存储、运行时上下文的读写操作） | 持久性（Persistent） |
| 生命周期上下文（Life-cycle Context） | 模型调用与工具执行之间发生的操作（如摘要生成、安全护栏、日志记录等） | 持久性（Persistent） |

- 临时上下文（Transient context）：指大语言模型（LLM）在单次调用中所看到的内容。你可以修改消息、工具或提示词，而无需改变状态中保存的数据。
- 持久上下文（Persistent context）：指在多轮交互中被保存到状态（state）中的内容。生命周期钩子（life-cycle hooks）和工具的写入操作会永久性地修改这一上下文。

> “临时” vs “持久” 的分界线不是“是否存到硬盘”，而是“是否跨越单次 LLM 调用”：
- Transient：只活在一次模型请求里。
- Persistent：活在整个智能体任务生命周期里（哪怕只是内存中的多轮对话）。

# 数据类型

在整个流程中，Agent会访问（读取/写入）不同类型的数据源：

| 数据源 | 别名 | 作用范围 | 示例 |
|--------|------|----------|------|
| 运行时上下文（Runtime Context） | 静态配置 | 单次对话范围内 | 用户 ID、API 密钥、数据库连接、权限信息、环境设置 |
| 状态（State） | 短期记忆 | 单次对话范围内 | 当前消息记录、上传的文件、认证状态、工具调用结果 |
| 存储（Store） | 长期记忆 | 跨对话（持久化） | 用户偏好设置、提取的洞察信息、记忆数据、历史记录 |

# 模型上下文

## system prompt

利用dynamic_prompt这个中间件，我们可以很容易根据运行时上下文动态修改系统提示。

### 根据state更改

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def state_aware_prompt(request: ModelRequest) -> str:
    # request.messages 是 request.state["messages"] 的快捷方式
    message_count = len(request.messages)

    base = "你是一个乐于助人的助手。"

    if message_count > 10:
        base += "\n这是一场很长的对话，请回答得格外简洁。"

    return base

agent = create_agent(
    model="gpt-4o",
    tools=[...],
    middleware=[state_aware_prompt]
)

### 根据store修改

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str  # 用户ID

@dynamic_prompt
def store_aware_prompt(request: ModelRequest) -> str:
    # 从运行时上下文中获取用户ID
    user_id = request.runtime.context.user_id

    # 读取存储：获取用户偏好
    store = request.runtime.store
    user_prefs = store.get(("preferences",), user_id)

    base = "你是一个乐于助人的助手。"

    if user_prefs:
        # 如果有用户偏好，获取沟通风格（默认为 balanced）
        style = user_prefs.value.get("communication_style", "balanced")
        base += f"\n用户偏好{style}风格的回答。"

    return base

# 创建智能体
agent = create_agent(
    model="gpt-4o",
    tools=[...],  # 工具列表
    middleware=[store_aware_prompt],  # 中间件：动态提示
    context_schema=Context,  # 上下文模式：定义需要的上下文数据结构
    store=InMemoryStore()  # 使用内存存储
)

### 根据context修改

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class Context:
    user_role: str        # 用户角色
    deployment_env: str   # 部署环境

@dynamic_prompt
def context_aware_prompt(request: ModelRequest) -> str:
    # 从运行时上下文中读取：用户角色和环境
    user_role = request.runtime.context.user_role
    env = request.runtime.context.deployment_env

    base = "你是一个乐于助人的助手。"

    if user_role == "admin":
        base += "\n你拥有管理员权限，可以执行所有操作。"
    elif user_role == "viewer":
        base += "\n你仅有只读权限，请仅引导用户进行读取操作。"

    if env == "production":
        base += "\n当前为生产环境，请格外小心任何数据修改操作。"

    return base

agent = create_agent(
    model="gpt-4o",
    tools=[...],
    middleware=[context_aware_prompt],
    context_schema=Context
)

## 消息列表

> 注意以下案例均使用wrap_model_call 实现临时更新—— 仅修改单次模型调用时发送给模型的消息内容，不会改变状态（state）中存储的信息。如果需要永久修改状态，需使用 before_model 或 after_model 等生命周期钩子。

### 根据state修改

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def inject_file_context(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """注入关于用户本会话上传文件的上下文信息。"""
    # 从状态中读取：获取已上传文件的元数据
    uploaded_files = request.state.get("uploaded_files", [])  

    if uploaded_files:
        # 构建可用文件的上下文描述
        file_descriptions = []
        for file in uploaded_files:
            file_descriptions.append(
                f"- {file['name']} ({file['type']}): {file['summary']}"
            )

        file_context = f"""本对话中你可以访问的文件：
{chr(10).join(file_descriptions)}

回答问题时请参考这些文件。"""

        # 在最近的消息前注入文件上下文
        messages = [  
            *request.messages,
            {"role": "user", "content": file_context},
        ]
        request = request.override(messages=messages)  

    return handler(request)

agent = create_agent(
    model="gpt-4o",
    tools=[...],
    middleware=[inject_file_context]
)

### 根据store修改

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str  # 用户ID

@wrap_model_call
def inject_writing_style(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """从存储中注入用户的邮件写作风格。"""
    user_id = request.runtime.context.user_id  

    # 从存储中读取：获取用户的写作风格示例
    store = request.runtime.store  
    writing_style = store.get(("writing_style",), user_id)  

    if writing_style:
        style = writing_style.value
        # 根据存储的示例构建风格指南
        style_context = f"""你的写作风格：
- 语气 (Tone): {style.get('tone', 'professional')}
- 常用问候语: "{style.get('greeting', 'Hi')}"
- 常用结束语: "{style.get('sign_off', 'Best')}"
- 你写过的邮件示例：
{style.get('example_email', '')}"""

        # 追加在消息末尾 - 模型通常更关注最后的消息
        messages = [
            *request.messages,
            {"role": "user", "content": style_context}
        ]
        request = request.override(messages=messages)  

    return handler(request)

agent = create_agent(
    model="gpt-4o",
    tools=[...],
    middleware=[inject_writing_style],
    context_schema=Context,
    store=InMemoryStore()
)

### 根据context修改

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@dataclass
class Context:
    user_jurisdiction: str  # 用户所属司法管辖区
    industry: str           # 所属行业
    compliance_frameworks: list[str]  # 合规框架列表

@wrap_model_call
def inject_compliance_rules(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """从运行时上下文中注入合规约束。"""
    # 从运行时上下文中读取：获取合规要求
    jurisdiction = request.runtime.context.user_jurisdiction  
    industry = request.runtime.context.industry  
    frameworks = request.runtime.context.compliance_frameworks  

    # 构建合规规则列表
    rules = []
    if "GDPR" in frameworks:
        rules.append("- 处理个人数据前必须获得明确同意")
        rules.append("- 用户拥有要求删除数据的权利")
    if "HIPAA" in frameworks:
        rules.append("- 未经授权不得共享患者健康信息")
        rules.append("- 必须使用安全加密的通信方式")
    if industry == "finance":
        rules.append("- 没有适当免责声明时，不得提供财务建议")

    if rules:
        # 生成合规上下文
        compliance_context = f"""{jurisdiction} 地区的合规要求：
{chr(10).join(rules)}"""

        # 追加在消息末尾 - 模型通常更关注最后的消息
        messages = [
            *request.messages,
            {"role": "user", "content": compliance_context}
        ]
        request = request.override(messages=messages)  

    return handler(request)

agent = create_agent(
    model="gpt-4o",
    tools=[...],
    middleware=[inject_compliance_rules],
    context_schema=Context
)

## Tools

这里只介绍根据state中用户的权限情况动态选择工具的案例，store和context的修改方式类似。

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def state_based_tools(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """根据对话状态过滤可用的工具。"""
    # 从状态中读取：检查用户是否已认证
    state = request.state  
    is_authenticated = state.get("authenticated", False)  
    message_count = len(state["messages"])

    # 仅在认证后启用敏感工具
    if not is_authenticated:
        # 过滤工具列表，仅保留名称以 "public_" 开头的公开工具
        tools = [t for t in request.tools if t.name.startswith("public_")]
        request = request.override(tools=tools)  
    elif message_count < 5:
        # 在对话初期限制工具使用（例如：禁用高级搜索）
        tools = [t for t in request.tools if t.name != "advanced_search"]
        request = request.override(tools=tools)  

    return handler(request)

agent = create_agent(
    model="gpt-4o",
    tools=[public_search, private_search, advanced_search],
    middleware=[state_based_tools]
)

## Models

本质上就是利用中间件来根据上下文动态选择模型，这点我们在自定义中间件章节的官方案例中已经介绍过了，这里就不再赘述了。

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

# 在中间件外部初始化模型（单例模式，避免重复初始化）
large_model = init_chat_model("claude-sonnet-4-5-20250929")
standard_model = init_chat_model("gpt-4o")
efficient_model = init_chat_model("gpt-4o-mini")

@wrap_model_call
def state_based_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """根据对话状态（消息数量）选择模型。"""
    # request.messages 是 request.state["messages"] 的快捷方式
    message_count = len(request.messages)  

    if message_count > 20:
        # 长对话 - 使用上下文窗口更大的模型
        model = large_model
    elif message_count > 10:
        # 中等长度对话
        model = standard_model
    else:
        # 短对话 - 使用高性价比模型
        model = efficient_model

    # 覆写请求中的模型
    request = request.override(model=model)  

    return handler(request)

agent = create_agent(
    model="gpt-4o-mini",  # 默认模型（会被中间件覆盖）
    tools=[...],
    middleware=[state_based_model]
)

## Response Format

根据state、store和context动态修改Agent的响应格式。

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from pydantic import BaseModel, Field
from typing import Callable

class SimpleResponse(BaseModel):
    """用于对话初期的简单响应格式。"""
    answer: str = Field(description="简短的回答")

class DetailedResponse(BaseModel):
    """用于深入对话的详细响应格式。"""
    answer: str = Field(description="详细的回答")
    reasoning: str = Field(description="推理过程")
    confidence: float = Field(description="置信度分数 0-1")

@wrap_model_call
def state_based_output(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """根据对话状态（消息数量）选择输出格式。"""
    # request.messages 是 request.state["messages"] 的快捷方式
    message_count = len(request.messages)  

    if message_count < 3:
        # 对话初期 - 使用简单格式
        request = request.override(response_format=SimpleResponse)  
    else:
        # 对话深入后 - 使用详细格式
        request = request.override(response_format=DetailedResponse)  

    return handler(request)

agent = create_agent(
    model="gpt-4o",
    tools=[...],
    middleware=[state_based_output]
)

# 工具上下文

工具的特殊之处在于它们既能读取上下文，也能写入上下文。我们之前提到过，工具可以从ToolRuntime中能够读取到当前的state、store和context。本小节主要介绍工具写入上下文的相关内容。


In [ ]:
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command
from typing import Annotated,Optional


class CustomState(AgentState):
    authenticated: bool = False 


@tool
def authenticate_user(password: str, runtime: ToolRuntime) -> Command:
    """验证用户并更新状态。"""
    # 模拟认证逻辑（实际应连接数据库/加密服务）
    is_correct = (password.strip().lower() == "secret123")
    
    if is_correct:
        print("[系统日志] 用户认证成功！")
        return Command(
            update={
                "messages": [ToolMessage(content="用户认证成功", tool_call_id=runtime.tool_call_id)],"authenticated": True
                }
            )
    else:
        print("[系统日志] 认证失败：密码错误")
        return Command(
            update={
                "messages": [ToolMessage(content="用户认证失败", tool_call_id=runtime.tool_call_id)],"authenticated": False
                }
            )

@tool
def check_balance(runtime: ToolRuntime) -> ToolMessage:
    """检查账户余额（仅限认证用户）。"""
    # 从运行时状态中读取认证信息
    state = runtime.state
    if not state.get("authenticated"):
        return {
            "messages": [ToolMessage(content="错误：请先使用 'authenticate_user' 工具进行身份验证。", tool_call_id=runtime.tool_call_id)]
        }
    else:
        return ToolMessage(content="您的账户余额为：$5,000.00", tool_call_id=runtime.tool_call_id)
    


agent = create_agent(
    model=model,
    tools=[authenticate_user, check_balance],
    state_schema=CustomState
)

In [18]:
for data in agent.stream(
    {"messages": [HumanMessage(content="密码secret123，我的余额是多少？")]},
    config={"configurable": {"thread_id": "1"}}
):
    for event, value in data.items():
        print(event, value["messages"][-1].content)


model 我来帮您验证用户并检查余额。
[系统日志] 用户认证成功！
tools 用户认证成功
model 认证成功！现在我来检查您的余额。
tools 您的账户余额为：$5,000.00
model 您的账户余额为：**$5,000.00**


# 生命周期上下文

案例见SummarizationMiddleware的用法